### BentoML Example
# Titanic Survival Prediction with XGBoost

This is a BentoML Demo Project demonstrating how to package and serve XGBoost model for production using BentoML.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment.

Let's get started!
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=xgboost&ea=xgboost-tiantic-survival-prediction&dt=xgboost-tiantic-survival-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install bentoml
!pip install xgboost numpy pandas

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import bentoml

# Prepare Dataset
download dataset from https://www.kaggle.com/c/titanic/data

In [4]:
!mkdir data
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv -o ./data/train.csv
!curl https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv -o ./data/test.csv

mkdir: data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60302  100 60302    0     0   621k      0 --:--:-- --:--:-- --:--:--  626k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28210  100 28210    0     0   505k      0 --:--:-- --:--:-- --:--:--  510k


In [5]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], label= train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

In [6]:
train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Survived']].head()

,Pclass,Age,Fare,SibSp,Parch,Survived
0,3,22.0,7.2500,1,0,0
1,1,38.0,71.2833,1,0,1
2,3,26.0,7.9250,0,0,1
3,1,35.0,53.1000,1,0,1
4,3,35.0,8.0500,0,0,0


# Model Training

In [7]:
params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

[14:19:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [10]:
y_test =  model.predict(X_test)
test['pred'] = y_test
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Create BentoService for model serving

In [11]:
%%writefile xgboost_titanic_bento_service.py

import xgboost as xgb

import bentoml
from bentoml.artifact import XgboostModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([XgboostModelArtifact('model')])
@bentoml.env(pip_dependencies=['xgboost'])
class TitanicSurvivalPredictionService(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler)
    def predict(self, df):
        data = xgb.DMatrix(data=df[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])
        return self.artifacts.model.predict(data)

Overwriting xgboost_titanic_bento_service.py


# Save BentoML service archive

In [12]:
# 1) import the custom BentoService defined above
from xgboost_titanic_bento_service import TitanicSurvivalPredictionService

# 2) `pack` it with required artifacts
bento_service = TitanicSurvivalPredictionService.pack(model = model)

# 3) save your BentoSerivce
saved_path = bento_service.save()

[2019-09-19 14:20:07,535] INFO - Successfully saved Bento 'TitanicSurvivalPredictionService:2019_09_19_a4ab8334' to path: /Users/chaoyuyang/bentoml/repository/TitanicSurvivalPredictionService/2019_09_19_a4ab8334


## Load saved BentoService for serving


In [13]:
import bentoml

bento_model = bentoml.load(saved_path)

result = bento_model.predict(test)
test['pred'] = result
test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch','pred']].iloc[10:].head(2)

[2019-09-19 14:20:07,771] WARNING - Module `xgboost_titanic_bento_service` already loaded, using existing imported module.
[14:20:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Pclass,Age,Fare,SibSp,Parch,pred
10,3,NaN,7.8958,0,0,0.341580
11,1,46.0,26.0000,0,0,0.413966


## Model Serving via REST API

In your termnial, run the following command to start the REST API server:

In [14]:
!bentoml serve {saved_path}

[14:20:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 * Serving Flask app "TitanicSurvivalPredictionService" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2019 14:20:14] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2019 14:20:16] "POST /predict HTTP/1.1" 200 -
^C


Copy following command to make a curl request to Rest API server

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[{"Pclass": 1, "Age": 30, "Fare": 200, "SibSp": 1, "Parch": 0}]' \
localhost:5000/predict
```